In [119]:
"""
Assignment 4
1.	Import training data provided in Assignment_4_Data_and_Template.xlsx 
2.	Use the pseudoinverse method on the given training data to build two linear classifiers: (i) 
a binary classifier, and (ii) a multiclass classifier capable of distinguishing between 6 classes. 
The sheet titled "Training Data" contains the target class labels (Failure and Type). 
Paste your classifiers in the correct format as shown in the sheet titled "Classifiers"
3.	Apply the classifiers to the data provided in the sheet "To be classified." 
Paste the results into the sheet.
4.	Evaluate the performance of your classifiers using the metrics requested in the sheet "Performance"
For guidance, see slides titled "Step-by-step: Linear Classifier Design" in the lecture on linear 
classifiers.
A completed spreadsheet is your submission. Except for writing results into the correct cells, 
do not alter the spreadsheets in any other way.
 
Extra Credit Opportunity: 
Flex your linear algebra muscles, mindfully!
Refer to the slide titled "What the pseudoinverse does"
•	In the context of Assignment 4, what does  wbest represent?
•	In the context of Assignment 4, what does  b represent?
•	Can you compute the vector  bp of Assignment 4?
•	Can you compute 5 random vectors ( b1,b2,b3,b4,b5) lying on the "Assignment 4 counterpart of the shaded 
plane" in the slide?
•	Using distance calculations, can you check that none of these random vectors are as close 
to  b as  bp is?
Use Assignment_4_Extra_Credit.xlsx  to submit your answers. Good luck and enjoy!
NOTE: Submitting a correct solution to this assignment will potentially get you a score of 125/100 for 
Assignment 4. [Similar to the situation in Assignment 2.] Please write to me if you need further 
clarification.

"""

'\nAssignment 4\n1.\tImport training data provided in Assignment_4_Data_and_Template.xlsx \n2.\tUse the pseudoinverse method on the given training data to build two linear classifiers: (i) \na binary classifier, and (ii) a multiclass classifier capable of distinguishing between 6 classes. \nThe sheet titled "Training Data" contains the target class labels (Failure and Type). \nPaste your classifiers in the correct format as shown in the sheet titled "Classifiers"\n3.\tApply the classifiers to the data provided in the sheet "To be classified." \nPaste the results into the sheet.\n4.\tEvaluate the performance of your classifiers using the metrics requested in the sheet "Performance"\nFor guidance, see slides titled "Step-by-step: Linear Classifier Design" in the lecture on linear \nclassifiers.\nA completed spreadsheet is your submission. Except for writing results into the correct cells, \ndo not alter the spreadsheets in any other way.\n \nExtra Credit Opportunity: \nFlex your linear a

In [120]:
import numpy as np
from matplotlib import pyplot as plt

def getSheetNames(excelfile):
    from pandas import ExcelFile
    return (ExcelFile(excelfile)).sheet_names

def readExcelSheet1(excelfile):
    from pandas import read_excel
    return (read_excel(excelfile)).values

def readExcelRange(excelfile, sheetname="Sheet1", startrow=1, endrow=1, startcol=1, endcol=1):
    from pandas import read_excel
    values = (read_excel(excelfile, sheetname, header=None)).values;
    return values[startrow - 1:endrow, startcol - 1:endcol]

def readExcel(excelfile, **args):
    if args:
        data = readExcelRange(excelfile, **args)
    else:
        data = readExcelSheet1(excelfile)
    if data.shape == (1, 1):
        return data[0, 0]
    elif (data.shape)[0] == 1:
        return data[0]
    else:
        return data
def writeExcelData(x, excelfile, sheetname, startrow, startcol, reshape=None):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    if reshape:
        df = DataFrame(x.reshape(-1, len(x)))
    else:
        df = DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname, startrow=startrow - 1, startcol=startcol - 1, header=False, index=False)
    writer.save()
    writer.close()


excelfile = r"/Users/omerorhan/Desktop/UCSC/MachineLearning/Assignments/venv/notebookfiles/" \
            r"Assignment#4/Assignment_4_Data_and_Template.xlsx";
sheets = getSheetNames(excelfile)
sheets

data = readExcel(excelfile)



In [130]:
def getClassifier(data):
#Collect and assemble 𝑁×𝑑 matrix of feature vectors into matrix 𝐗. 
    X = data[:,:-4]
    #Assemble targets into 𝐓
    F = np.array(data[:, 15], dtype=int)
    Tnum =  np.array(data[:,-3:-2], dtype=int)
    N = len(X)
    w = np.ones(N)
    w[0] = 0 
    ones = np.ones((N,1),dtype=int)
    #Construct the augmented feature vector set 𝐗𝐚 by prepending a column of 1s in front of the matrix 𝐗.
    # Note that 𝐗𝐚 has dimensions 𝑁×(𝑑+1).
    Xa = np.concatenate((ones,X), axis=1)
    print(Xa.shape)
    XaPI = np.linalg.pinv(Xa.astype(int))
    print(XaPI.shape)
    print(F.shape)
    wF= np.matmul(XaPI,F)
    classes = 6
    T = np.full((N, classes), -1, dtype=int)
    for i in range(N):
        for c in range(classes):
            if (Tnum[i] == c):
                T[i, c] = 1
                break
    wT = np.matmul(XaPI, T)
    return wF, wT, X, F, T, N
wF, wT, Xtrain, Ftrain, Typetrain, Ntrain = getClassifier(data)
writeExcelData(wF, excelfile, "Classifiers", 5, 1, reshape=None)
writeExcelData(wT, excelfile, "Classifiers", 5, 5, reshape=None)
print(Typetrain.shape)
testdata = readExcelRange(excelfile,sheetname=sheets[2],startrow=5,endrow=54,startcol=1,endcol=15)


(6600, 16)
(16, 6600)
(6600,)


(6600, 6)


In [153]:
def classify(X, wF, wT):
    N = np.size(X, axis=0)
    print("classify N %d" % (N))
    ones = np.ones((N, 1), dtype=int)
    Xa = np.concatenate((ones, X), axis=1)
    print(wF.shape)
    Failure = np.dot(Xa, wF)
    print(Failure.shape)
    F = np.zeros(N, dtype=int)
    print(F.shape)
    indicatesNegative = Failure < 0
    indicatesPositive = Failure > 0
    F[indicatesNegative] = -1
    F[indicatesPositive] = 1
    T = np.dot(Xa, wT)
    classes = 6
    Tnum = np.zeros(N, dtype=int)
    for i in range(N):
        max = T[i, 0]
        maxc = 0
        for c in range(classes):
            if (T[i, c] > max):
                max = T[i, c]
                maxc = c
        Tnum[i] = maxc
    return F, Tnum
Ftest, Ttest = classify(testdata, wF, wT)

writeExcelData(Ftest, excelfile, sheets[2], 5, 16)
writeExcelData(Ttest, excelfile, sheets[2], 5, 17)




classify N 50
(16,)
(50,)
(50,)


In [155]:
Ftrain_class, Typetrain_class = classify(Xtrain, wF, wT)

TP = 0  # TFail
TN = 0  # TPass
FP = 0  # FFail
FN = 0  # FPass

for i in range(Ntrain):
if (Ftrain[i] == Ftrain_class[i]):
    if (Ftrain[i] == 1):
        TP = TP + 1
    else:
        TN = TN + 1
else:
    if (Ftrain[i] == 1):
        FN = FN + 1
    else:
        FP = FP + 1

writeExcelData([TP], excelfile, sheets[3], 10, 3)
writeExcelData([FN], excelfile, sheets[3], 10, 4)
writeExcelData([FP], excelfile, sheets[3], 11, 3)
writeExcelData([TN], excelfile, sheets[3], 11, 4)

accuracy = TP + FN / (TP + FN + TN + FP)
writeExcelData([accuracy], excelfile, sheets[3], 8, 7)

sensivity = TP/(TP+FN)
writeExcelData([accuracy], excelfile, sheets[3], 9, 7)

specifity = TN/(FP+TN)
writeExcelData([accuracy], excelfile, sheets[3], 10, 7)

ppvB = TP/(FP+TP)
writeExcelData([ppvB], excelfile, sheets[3], 11, 7)
classes = 6 
confusionmatrix = np.zeros((classes,classes),dtype=int)

for n in range(Ntrain):
truth = Typetrain[n]
result = Typetrain_class[n]
if (truth >= classes or truth < 0):
    print("Truth %d not in range of classes" % (truth))
if (result >= classes or result < 0):
    print("Result %d not in range of classes" % (result))
confusion[truth, result] = confusion[truth, result] + 1

writeExcelData(confusion, excelfile, sheets[3], 19, 3)
cmax = 0
maxppv = 0
cmin = 0
minppv = 1
for c in range(classes):
    sum = 0
    for r in range(classes):
        sum = sum + confusion[r,c]
    ppv = confusion[c,c]/sum
    print("Class %d: confusion[c,c] %f, sum %f -> PPV %f" % (c,confusion[c,c],sum,ppv))
    if (ppv > maxppv):
        maxppv = ppv
        cmax = c
    if (ppv < minppv):
        minppv = ppv
        cmin = c

writeExcelData([maxppv], excelfile, sheets[3], 20, 12)
writeExcelData([cmax], excelfile, sheets[3], 20, 13)
writeExcelData([minppv], excelfile, sheets[3], 21, 12)
writeExcelData([cmin], excelfile, sheets[3], 21, 13)


classify N 6600
(16,)
(6600,)
(6600,)
